In [ ]:
import glob

import matplotlib.pyplot as plt

plt.style.use("default")
import numpy as np
import tensorflow as tf
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split

from src.classes import Boundary
from src.performance_evaluation import (
    center_of_mass,
    compute_position_error,
    compute_volume_error,
    compute_voxel_error,
)
from src.util import plot_voxel, plot_voxel_c, voxel_ball

print(tf.__version__)

import os

import pandas as pd
import seaborn as sns

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

boundary = Boundary()

In [ ]:
from dataclasses import dataclass


@dataclass
class BallAnomaly:  #  FOR VAE TRAINING ONLY
    x: float
    y: float
    z: float
    d: float
    γ: float

# Stage 1

Build the variational autoencoder $\mathbb{VAE}$ to learn a latent representation $\mathbf{z}$ of the domain $\Omega \in \mathbb{R}^3$.

Object radius

- 20mm $\rightarrow$ `d = 4`
- 30mm $\rightarrow$ `d = 6`
- 40mm $\rightarrow$ `d = 8`

In [ ]:
ball = BallAnomaly(x=10.5, y=10, z=10, d=4, γ=1)

In [ ]:
ball_vxl = voxel_ball(ball, boundary)

In [ ]:
plot_voxel(ball_vxl)

In [ ]:
X_all_anomalys = list()
diameter_labels = list()
pos_ball = list()
vol_ball = list()

max_d = 8

for γ in [1]:  # set object geometries to 1 and empty space to 0
    for d in [4, 6, 8]:  # diameter
        for x in np.linspace(boundary.x_0 + max_d, boundary.x_length - max_d, 15):
            for y in np.linspace(boundary.y_0 + max_d, boundary.y_length - max_d, 15):
                for z in np.linspace(
                    boundary.z_0 + max_d, boundary.z_length - max_d, 15
                ):
                    ball = BallAnomaly(x, y, z, d, γ)
                    vxl_ball = voxel_ball(ball, boundary)

                    X_all_anomalys.append(vxl_ball)
                    diameter_labels.append(d)
                    pos_ball.append([ball.y, ball.x, ball.z])  #  y,x,z in 3d Plot!!!
                    vol_ball.append(np.where(vxl_ball == 1)[0].shape[0])

X_all_anomalys = np.array(X_all_anomalys) / 2
diameter_labels = np.array(diameter_labels)
pos_ball = np.array(pos_ball)
vol_ball = np.array(vol_ball)

In [ ]:
print(X_all_anomalys.shape, diameter_labels.shape)

In [ ]:
(
    X_train,
    X_test,
    r_train,
    r_test,
    pos_train,
    pos_test,
    vol_train,
    vol_test,
) = train_test_split(
    X_all_anomalys,
    diameter_labels,
    pos_ball,
    vol_ball,
    train_size=0.9,
    shuffle=True,
    random_state=42,
)

In [ ]:
print(
    X_train.shape,
    X_test.shape,
    r_train.shape,
    r_test.shape,
    pos_train.shape,
    pos_test.shape,
    vol_train.shape,
    vol_test.shape,
)

In [ ]:
for rdn in np.random.randint(low=0, high=X_train.shape[0], size=5):
    print(rdn)
    print("Pos:\t", pos_train[rdn])
    print("Vol:\t", vol_train[rdn])
    plot_voxel_c(X_train[rdn, ...] * 2)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.activations import sigmoid
from tensorflow.keras.backend import random_normal
from tensorflow.keras.layers import (
    BatchNormalization,
    Conv3D,
    Conv3DTranspose,
    Dense,
    Flatten,
    Input,
    Layer,
    Reshape,
)
from tensorflow.keras.losses import (
    binary_crossentropy,
    mean_absolute_error,
    mean_squared_error,
)
from tensorflow.keras.metrics import Mean
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD, Adam

filters = [1, 2, 4, 8]
kernels = [(5, 5, 5) for _ in range(4)]
strides = [(1, 1, 1), (2, 2, 2), (2, 2, 2), (2, 2, 2)]
paddings = ["same", "same", "same", "same"]

latent_dim = 8


class Sampling(Layer):
    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon


class VAE(Model):
    def __init__(self, encoder, decoder, beta=1.0, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.beta = beta
        self.total_loss_tracker = Mean(name="total_loss")
        self.reconstruction_loss_tracker = Mean(name="reconstruction_loss")
        self.kl_loss_tracker = Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
            self.total_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            kl_loss = 1 + z_log_var - tf.pow(z_mean, 2) - tf.exp(z_log_var)
            kl_loss = -0.5 * tf.reduce_sum(kl_loss, axis=-1)
            reconstruction_loss = binary_crossentropy(
                data, reconstruction, axis=(1, 2, 3)
            )  #
            reconstruction_loss *= np.prod((32, 32, 32, 1))
            # β-VAE
            # print("beta value:",self.beta)
            total_loss = reconstruction_loss + self.beta * kl_loss

        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
            "total_loss": self.total_loss_tracker.result(),
        }

    def get_config(self):
        config = super(VAE, self).get_config()
        config.update(
            {
                "encoder": tf.keras.utils.serialize_keras_object(self.encoder),
                "decoder": tf.keras.utils.serialize_keras_object(self.decoder),
                "beta": self.beta,
            }
        )
        return config


def encoder_model(
    input_shape=(32, 32, 32, 1),
    filters=filters,
    kernels=kernels,
    strides=strides,
    paddings=paddings,
    latent_dim=latent_dim,
):
    encoder_inputs = Input(shape=input_shape)
    x = BatchNormalization()(encoder_inputs)

    for fltr, krnl, strd, pddng in zip(filters, kernels, strides, paddings):
        x = Conv3D(
            filters=fltr,
            kernel_size=krnl,
            strides=strd,
            padding=pddng,
            activation="elu",
        )(x)
        x = BatchNormalization()(x)

    x = Flatten()(x)

    z_mean = Dense(latent_dim, name="z_mean", activation="sigmoid")(x)
    z_log_var = Dense(latent_dim, name="z_log_var")(x)
    z = Sampling()((z_mean, z_log_var))

    return encoder_inputs, z_mean, z_log_var, z


def decoder_model(
    input_shape=(32, 32, 32, 1),
    filters=filters[::-1],
    kernels=kernels[::-1],
    strides=strides[::-1],
    paddings=paddings[::-1],
    latent_dim=latent_dim,
):
    latent_inputs = Input(shape=(latent_dim,))
    x = Dense(512, activation="relu")(latent_inputs)
    x = Reshape((4, 4, 4, 8))(x)

    for fltr, krnl, strd, pddng in zip(filters, kernels, strides, paddings):
        x = Conv3DTranspose(
            filters=fltr,
            kernel_size=krnl,
            strides=strd,
            padding=pddng,
            activation="elu",
        )(x)
        x = BatchNormalization()(x)

    decoded = x

    return latent_inputs, decoded


def vae_model(
    input_shape=(32, 32, 32, 1),
    filters=filters,
    kernels=kernels,
    strides=strides,
    paddings=paddings,
    latent_dim=latent_dim,
    beta=1.0,
):
    encoder_inputs, z_mean, z_log_var, z = encoder_model(
        input_shape=(32, 32, 32, 1),
        filters=filters,
        kernels=kernels,
        strides=strides,
        paddings=paddings,
        latent_dim=latent_dim,
    )
    encoder = Model(encoder_inputs, (z_mean, z_log_var, z), name="VAE_encoder")

    decoder_inputs, decoder_outputs = decoder_model(
        input_shape=(32, 32, 32, 1),
        filters=filters[::-1],
        kernels=kernels[::-1],
        strides=strides[::-1],
        paddings=paddings[::-1],
        latent_dim=latent_dim,
    )
    decoder = Model(decoder_inputs, decoder_outputs, name="VAE_decoder")

    return VAE(encoder, decoder, beta=beta)


# engineering decoder and encoder parts:

encoder_inputs, z_mean, z_log_var, z = encoder_model(
    input_shape=(32, 32, 32, 1),
    filters=filters,
    kernels=kernels,
    strides=strides,
    paddings=paddings,
    latent_dim=latent_dim,
)
encoder = Model(encoder_inputs, (z_mean, z_log_var, z), name="VAE_encoder")

decoder_inputs, decoder_outputs = decoder_model(
    input_shape=(32, 32, 32, 1),
    filters=filters[::-1],
    kernels=kernels[::-1],
    strides=strides[::-1],
    paddings=paddings[::-1],
    latent_dim=latent_dim,
)
decoder = Model(decoder_inputs, decoder_outputs, name="VAE_decoder")

encoder.summary()
decoder.summary()

In [ ]:
# |--------------------------------------------------------- HPT -------------------------------------------------------|
# | model  1: epochs = 500, batch_size = 128, beta = 1.05, kernels = (4, 4, 4),filters = [1, 2,  4,  8], latent_dim = 8 |
# | model  2: epochs = 500, batch_size = 128, beta = 1.04, kernels = (4, 4, 4),filters = [1, 2,  4,  8], latent_dim = 8 |
# | model  3: epochs = 500, batch_size = 128, beta = 1.03, kernels = (4, 4, 4),filters = [1, 2,  4,  8], latent_dim = 8 |
# | model  4: epochs = 500, batch_size = 128, beta = 1.02, kernels = (4, 4, 4),filters = [1, 2,  4,  8], latent_dim = 8 |
# | model  5: epochs = 500, batch_size = 128, beta = 1.01, kernels = (4, 4, 4),filters = [1, 2,  4,  8], latent_dim = 8 |
# |                                                                                            |
# | model  6: epochs = 500, batch_size = 128, beta = 1.00, kernels = (4, 4, 4),filters = [1, 2,  4,  8], latent_dim = 8 |
# | model  7: epochs = 500, batch_size = 128, beta = 0.99, kernels = (4, 4, 4),filters = [1, 2,  4,  8], latent_dim = 8 |
# | model  8: epochs = 500, batch_size = 128, beta = 0.98, kernels = (4, 4, 4),filters = [1, 2,  4,  8], latent_dim = 8 |
# | model  9: epochs = 500, batch_size = 128, beta = 0.97, kernels = (4, 4, 4),filters = [1, 2,  4,  8], latent_dim = 8 |
# | model 10: epochs = 500, batch_size = 128, beta = 0.96, kernels = (4, 4, 4),filters = [1, 2,  4,  8], latent_dim = 8 |
# |                                                                                            |
# | model 11: epochs = 500, batch_size = 128, beta = 1.04, kernels = (3, 3, 3),filters = [1, 2,  4,  8], latent_dim = 8 |
# | model 12: epochs = 500, batch_size = 128, beta = 1.04, kernels = (5, 5, 5),filters = [1, 2,  4,  8], latent_dim = 8*|
# | model 13: epochs = 500, batch_size = 128, beta = 1.04, kernels = (6, 6, 6),filters = [1, 2,  4,  8], latent_dim = 8 |
# | model 14: epochs = 500, batch_size = 128, beta = 1.04, kernels = (7, 7, 7),filters = [1, 2,  4,  8], latent_dim = 8 |
# | model 15: epochs = 500, batch_size = 128, beta = 1.04, kernels = (9, 9, 9),filters = [1, 2,  4,  8], latent_dim = 8 |
# |                                                                                            |
# | model 16: epochs = 500, batch_size =  32, beta = 1.04, kernels = (5, 5, 5),filters = [1, 2,  4,  8], latent_dim = 8 |
# | model 17: epochs = 500, batch_size =  64, beta = 1.04, kernels = (5, 5, 5),filters = [1, 2,  4,  8], latent_dim = 8 |
# | model 18: epochs = 500, batch_size = 128, beta = 1.04, kernels = (5, 5, 5),filters = [1, 2,  4,  8], latent_dim = 8 |
# | model 19: epochs = 500, batch_size = 256, beta = 1.04, kernels = (5, 5, 5),filters = [1, 2,  4,  8], latent_dim = 8 |
# | model 20: epochs = 500, batch_size = 512, beta = 1.04, kernels = (5, 5, 5),filters = [1, 2,  4,  8], latent_dim = 8 |
# |                                                                                            |
# | model 21: epochs = 500, batch_size =  32, beta = 1.04, kernels = (5, 5, 5),filters = [1, 2,  4,  8], latent_dim = 8 |
# | model 22: epochs = 500, batch_size =  16, beta = 1.04, kernels = (5, 5, 5),filters = [1, 2,  4,  8], latent_dim = 8 |
# | model 23: epochs = 500, batch_size =   8, beta = 1.04, kernels = (5, 5, 5),filters = [1, 2,  4,  8], latent_dim = 8 |
# | model 24: epochs = 200, batch_size =   4, beta = 1.04, kernels = (5, 5, 5),filters = [1, 2,  4,  8], latent_dim = 8 |
# | model 25: epochs = 200, batch_size =   2, beta = 1.04, kernels = (5, 5, 5),filters = [1, 2,  4,  8], latent_dim = 8 |
# |---------------------------------------------------------------------------------------------------------------------|

# learning_rate = 0.0001
# sgd = SGD(learning_rate = learning_rate_1, momentum = 0.9, nesterov = True)
# vae.compile(optimizer = SGD())# , metrics = ['accuracy']

epochs = 500
batch_size = 32
beta = 1.04  #  best value from hpt

cb = tf.keras.callbacks.EarlyStopping(monitor="loss", patience=0, start_from_epoch=47)

vae = vae_model(input_shape=(32, 32, 32, 1), filters=[1, 4, 8, 16], beta=beta)
vae.compile(optimizer=Adam())  # learning_rate = learning_rate

history = vae.fit(
    np.expand_dims(X_train, 4),
    epochs=epochs,
    batch_size=batch_size,
    callbacks=[cb],
)

In [ ]:
for keys in history.history.keys():
    plt.plot(history.history[keys], label=keys)
    plt.legend()
plt.show()

In [ ]:
vae_idx = len(glob.glob("models/vaes/*.h5")) + 1
vae.save_weights(f"models/vaes/vae_{vae_idx}.weights.h5")

In [ ]:
# vae_idx = 2
# vae.load_weights(f"models/vaes/vae_{vae_idx}.weights.h5")

In [ ]:
plt.style.use("default")
for rdn in np.random.randint(low=0, high=X_test.shape[0], size=5):
    print("Original...")
    plot_voxel_c(X_test[rdn, ...])

    print("Prediction...")
    _, _, z = vae.encoder.predict(np.expand_dims(X_test[rdn, ...], 0))
    sgl_pred = vae.decoder.predict(z)
    sgl_pred = np.squeeze(sgl_pred, axis=4)
    sgl_pred = np.squeeze(sgl_pred, axis=0)
    sgl_pred = np.clip(sgl_pred, a_min=0, a_max=1)
    plot_voxel_c(sgl_pred)

In [ ]:
acc_measure = dict(Model=list(), Error_val=list(), Error=list(), test_idx=list())
# Apply test data to previously trained VAE
_, _, z = vae.encoder.predict(X_test)
X_pred = vae.decoder.predict(z)
X_pred = np.squeeze(X_pred, axis=4)
X_pred = np.clip(X_pred, a_min=0, a_max=1)
# X_pred = np.round(X_pred, 1) + 0.15  #  round offset
# X_pred = np.round(X_pred)


data_append = []

for idx, v_err in enumerate(compute_volume_error(X_pred, X_test)):
    v_percent = (v_err / 32**3) * 100
    data_append.append(
        dict(Model=vae_idx, Error_val=v_percent, Error="Volume", test_idx=idx)
    )

for idx, p_err in enumerate(compute_position_error(pos_test, X_pred)):
    p_percent = (p_err / np.linalg.norm(np.array([32, 32, 32]))) * 100
    data_append.append(
        dict(Model=vae_idx, Error_val=p_percent, Error="Position", test_idx=idx)
    )

# Iterativ das Dictionary erweitern
for item in data_append:
    for key, value in item.items():
        acc_measure[key].append(value)

In [ ]:
acc_measure_df = pd.DataFrame(acc_measure).dropna()
acc_measure_df.to_csv(f"models/vaes/vae_acc_measure_{vae_idx}.csv", index=False)
acc_measure_df

In [ ]:
custom_params = {"axes.spines.right": False, "axes.spines.top": False}
sns.set_theme(style="ticks", rc=custom_params)
sns.set_context(context="paper", font_scale=1.4)

sns.boxplot(
    x="Model", y="Error_val", data=acc_measure_df, hue="Error", showfliers=False
)  #  violinplot: , split=True
plt.ylabel("Error (%)")
# plt.xlabel("Model Iteration")

**Append Stats**

In [ ]:
custom_params = {"axes.spines.right": False, "axes.spines.top": False}
sns.set_theme(style="ticks", rc=custom_params)
sns.set_context(context="paper", font_scale=1.4)

combined_df = pd.concat(
    [pd.read_csv(f_csvs) for f_csvs in glob.glob("models/vaes/vae_acc_measure_*.csv")],
    ignore_index=True,
)
combined_df = pd.DataFrame(combined_df)
# combined_df.to_csv(f"models/vaes/all_measures.csv", index=False)
custom_palette = ["#756bb1", "#a1d99b"]
sns.boxplot(
    x="Model",
    y="Error_val",
    data=combined_df,
    hue="Error",
    showfliers=False,
    palette=custom_palette,
)  #  violinplot: , split=True
plt.ylabel("Error (%)")
# plt.vlines(16 - 1, -0.2, 2, colors="#e34a33", linestyle="dashed")
plt.vlines(17 - 1, -0.2, 2, colors="#fec44f", linestyle="dashed")
plt.vlines(21 - 1, -0.2, 2, colors="#31a354", linestyle="dashed")
plt.tight_layout()
plt.xlim([-0.5, 25])
plt.savefig("images/vae_hpt.pdf")
plt.savefig("images/vae_hpt.png")
plt.show()

___

## Further evaluation